# Linear Regression

Author: Sampath K Perera and  Kajhanan Kailainathan

This notebook is based on [this](https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html) scikit learn tutorial.

## Import the Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection as  model_selection
#load the dataset
from sklearn.datasets import fetch_california_housing
import pandas as pd

## Visualize the dataset

Fetch the California Housing Dataset through the *datasets* module of the *sklearn* library

In [ ]:
california_housing = fetch_california_housing(as_frame=True)

In [ ]:
type(california_housing.data)

The entire tabular dataset could be accessed using the .data attribute. The .head() method is used to visualize a compact dataframe of only 5 entries. This can be used to visualize the independent variable values and their names.

In [ ]:
california_housing.data.head()

The depend variable name and values could be accessed through the .target attribute.

In [ ]:
# print dependent variable (y)
california_housing.target.head()

The .frame attribute contains a Dataframe of both the dependent and independent variables. The .info() method gives the Column name, Non-Null count (number of entries with non-null values), and Data Type. Verify by running the code snippet below. Do try out the .info() method on .target and .data attributes as well.

In [ ]:
california_housing.frame.info()

The .hist() method can be called upon a dataframe to plot histogram distribution for each column variables. This can be used to get an idea of the distribution of each of the variables in an independent manner.

In [ ]:
california_housing.frame.hist(figsize=(12, 10), bins=30, edgecolor="black")
plt.subplots_adjust(hspace=0.7, wspace=0.4)

## Data Analysis (Checking for Outliers)

From the above variables, the histogram plots of the 'Average Rooms', 'Average Bedrooms', 'Average Population', and 'Population' seem to have a significant skew. (i.e), the data is bulged within a short range of the entire range of the data distribution.

This points at a potential source of outliers.

The .describe() method can be called upon the selected features, which gives out the mean, standard deviation, inter quartile range values, and other significant stastical measures which can be used to analyse the data.

In [ ]:
features_of_interest = ["AveRooms", "AveBedrms", "AveOccup", "Population"]
california_housing.frame[features_of_interest].describe()

A large gap on the 75th percentile value and the maximum value indicates possible outliers.

## Linear Regression on a Single Variable

Let us perform linear regression on a single variable with potential outliers. We will compare different methods and their performance at the presence of outliers.

In [ ]:
features_of_interest = ["AveRooms"]
california_housing.frame[features_of_interest].describe()

In [ ]:
# Load the  dataset
data_X, data_y = fetch_california_housing(return_X_y=True)

# select the "AveRooms" data
data_X=data_X[:,2]

#save index of the maximum value, will be used later
index_max=np.argmax(data_X)
max_x=data_X[index_max]
max_y=data_y[index_max]

In [ ]:
#verification that we have selected correct set
print(np.max(data_X),np.min(data_X),np.mean(data_X))

Select only 100 datapoints at random and perform regression

In [ ]:
#we have lot of data samples, just select 100 of them
data_X=data_X[100:200]
data_y=data_y[100:200]
#remember index starts from 0
# Split the data into training/testing sets
X_train, X_test, y_train, y_test = model_selection.train_test_split(data_X, data_y, test_size=0.2, random_state=0)

X_train, X_val, y_train, y_val  = model_selection.train_test_split(X_train, y_train, test_size=0.25, random_state=0)

print("Total dataset elements",data_X.shape)
print("Train dataset elements",X_train.shape)
print("Validatiom dataset elements",X_val.shape)
print("Test dataset elements",X_test.shape)

Most sklearn methods (and also methods from other machine learning libraries), deal with tensors. The above shape (100,) for instance refers to a numpy array of 100 scalar values. However, we must convert this to a numpy array of 100 1 dimensional tensors.

**This intermediate step will be commonly encountered even when working with other Machine Learning libraries, thus is helpful to make a note on this if you are already not aware**

It is encouraged that you visualize the shapes of the X_train and other variables before and after applying the .reshape() method using the .shape attribute

In [ ]:
#reshape data as vectors
# print(Shape before reshaping)
# print(X_train.shape)
X_train = X_train.reshape(X_train.shape[0], 1)
# print(Shape after reshaping)
# print(X_train.shape)
X_test = X_test.reshape(X_test.shape[0], 1)
X_val = X_val.reshape(X_val.shape[0], 1)
y_train = y_train.reshape(y_train.shape[0], 1)
y_test = y_test.reshape(y_test.shape[0], 1)
yval = y_val.reshape(y_val.shape[0], 1)

The *linear_model.LinearRegression()* module from the *sklearn* library can be used to perform linear regression and tabulate the corresponding variables

In [ ]:
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)


# The intercept
print("Intercept w_0: \n", regr.intercept_)
# The coefficients
print("Coefficient w_1: \n", regr.coef_)



Instead of using the *linear_model.LinearRegression()* module, the known equation $\hat{\beta} = (X^T X)^{-1}X^T y$ can be directly applied using numpy methods.

Verify that both the answers are the same

In [ ]:
import numpy as np



# Reshape y_train to ensure it's a column vector
y_train = y_train.reshape(-1, 1)

# Add a column of 1s to X_train for the intercept term
intercept = np.ones((X_train.shape[0], 1))
X_train_with_intercept = np.hstack((intercept, X_train))

# Calculate beta_hat using the normal equation
X_transpose = X_train_with_intercept.T
X_transpose_X = np.dot(X_transpose, X_train_with_intercept)
X_transpose_X_inv = np.linalg.inv(X_transpose_X)
X_transpose_y = np.dot(X_transpose, y_train)
beta_hat = np.dot(X_transpose_X_inv, X_transpose_y)

# Extract the intercept and coefficients
intercept_hat = beta_hat[0]
coefficients = beta_hat[1:]

print("Intercept w_0 :", intercept_hat)
print("Coefficient w_1:", coefficients)


The variables RSS (Residual Sum of Squares), RSE (Residual Standard Error), and TSS (Total Sum of Squares) can be tabulated as shown below by directly applying numpy methods. The predicted values are obtained using the .predict() method from the instantiation of linear_model.LinearRegression().

In [ ]:
yhat=regr.predict(X_train)

# Residual Sum of Sqaures (RSS)
RSS = np.sum((yhat - y_train)**2)
print('RSS=', RSS)

N=len(y_train)
print('Number of Datapoints=',N)

# Residual Standard Error (RSE)
RSE = np.sqrt(1/(N-2)*RSS)
print('RSE=', RSE)

# Total Sum of Squares (TSS)
TSS = np.sum((y_train- np.mean(y_train))**2)
print('TSS=', TSS)

R2 = (TSS - RSS)/TSS
print('R2 (from direct calculations)=', R2)

# Calculation of R2 using sklearn
R2 = regr.score(X_train,y_train)

print('R2 (from sklearn module)=', R2)

Calculate the t-statistic and p-values

In [ ]:
from scipy.stats import t


w_1=regr.coef_
w_0=regr.intercept_
sigma2 = np.var(y_train - yhat)
SE2w0 =  sigma2*(1/N + np.mean(X_train)**2/np.sum((X_train - np.mean(X_train))**2))
SE2w1 = sigma2/np.sum((X_train - np.mean(X_train))**2)
tw1 = (w_1 - 0)/np.sqrt(SE2w1)
tw0 = (w_0 - 0)/np.sqrt(SE2w0)

print('Standard errors for intecept and w1: ', SE2w0, SE2w1)
print('t-statistic for intecept and w1: ', tw0, tw1)
pw1 = t.sf(np.abs(tw1), N-2)

pw0 = t.sf(np.abs(tw0), N-2)
print('p-value  for intecept and w1: ', pw0, pw1)

Use the calculated regression coefficients, and check its validity,
1. Quantitatively : Using Mean squared error and Coefficient of Determination
2. Qualitatively : By plotting and visualizing the test datapoints as a scatter plot against the regression line

In [ ]:
# Make predictions using the testing set
y_pred = regr.predict(X_test)



# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

# Plot outputs
plt.scatter(X_test, y_test, color="black", label='Data points')
plt.plot(X_test, y_pred, color="blue", linewidth=2, label=r'Linear Regression')


plt.legend(loc='best')
plt.xlabel('AveRooms')
plt.ylabel('MedHouseVal')
plt.ylim((0,5))
plt.xlim((0,10))
plt.title("Testing data")
plt.show()

The coeffeicient of determination value is well above 0.5. This means that the regression method has worked to some length. However, we should also consider the fact that outliers could have been removed when we took a random sample of the training set to determine the regression coefficients.

Let us observe how the presence of outliers impacts the regression method.

## Regression with Outliers

Perform regression after including the maximum x and corresponding y value

In [ ]:
# load maximum x value and corresponding y value to the training dataset
X_train[59,:]=max_x
y_train[59,:]=max_y
# Train the model using the training sets
regr.fit(X_train, y_train)
# The coefficients
print("Coefficients: \n", regr.coef_)
print("intercept: \n", regr.intercept_)

#see how training line look like


# Plot outputs
plt.scatter(X_train, y_train, color="black", )
plt.plot(X_train,  regr.predict(X_train), color="blue", linewidth=2)

plt.subplots(1, 1)
plt.scatter(X_train, y_train, color="black", label='Data points')
plt.plot(X_train,  regr.predict(X_train), color="blue", linewidth=2, label=r'Linear Regression')


plt.legend(loc='best')
plt.xlabel('AveRooms')
plt.ylabel('MedHouseVal')
plt.title("Training data")
plt.show()
plt.subplots(1, 1)
plt.scatter(X_train, y_train, color="black", label='Data points')
plt.plot(X_train,  regr.predict(X_train), color="blue", linewidth=2, label=r'Linear Regression')


plt.legend(loc='best')
plt.xlabel('AveRooms')
plt.ylabel('MedHouseVal')
plt.ylim((0,5))
plt.xlim((0,10))
plt.title("Training data (zoom)")
plt.show()


# The coefficients
print("Coefficients: \n", regr.coef_)
# The intercept
print("Coefficients: \n", regr.intercept_)

See the predictions using test set

In [ ]:
y_pred = regr.predict(X_test)


# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))




# Plot outputs
plt.scatter(X_test, y_test, color="black", label='Data points')
plt.plot(X_test, y_pred, color="blue", linewidth=2, label=r'Linear Regression')


plt.legend(loc='best')
plt.xlabel('AveRooms')
plt.ylabel('MedHouseVal')
plt.ylim((0,5))
plt.xlim((0,10))
plt.title("Testing data")
plt.show()

## RANSAC (RANdom SAmple Consensus)

RANSAC is an iterative parameter estimation method that performs well even at the presence of outliers. Click [this](http://www.cse.yorku.ca/~kosta/CompVis_Notes/ransac.pdf) to learn more about RANSAC.

*linear_model.RANSACRegressor()* from *sklearn* is used to implement the RANSAC algorithm.

In [ ]:
# Train the model using the training sets
ransac = linear_model.RANSACRegressor()
#The RANSAC regressor automatically splits the data into inliers and outliers, and the fitted line is determined only by the identified inliers.

ransac.fit(X_train, y_train)

# Plot outputs
plt.subplots(1, 1)
plt.scatter(X_train, y_train, color="black", label='Data points')
plt.plot(X_train,  regr.predict(X_train), color="blue", linewidth=2, label=r'Linear Regression')
plt.plot(X_train, ransac.predict(X_train), color="red", linewidth=2, label=r'RANSAC Regressor')

plt.legend(loc='best')
plt.xlabel('AveRooms')
plt.ylabel('MedHouseVal')
plt.title("Training data")
plt.show()
plt.subplots(1, 1)
plt.scatter(X_train, y_train, color="black", label='Data points')
plt.plot(X_train,  regr.predict(X_train), color="blue", linewidth=2, label=r'Linear Regression')
plt.plot(X_train, ransac.predict(X_train), color="red", linewidth=2, label=r'RANSAC Regressor')

plt.legend(loc='best')
plt.xlabel('AveRooms')
plt.ylabel('MedHouseVal')
plt.ylim((0,5))
plt.xlim((0,10))
plt.title("Training data (zoom)")
plt.show()

Predict values with test data

In [ ]:
line_y_ransac = ransac.predict(X_test)

# Plot outputs
plt.scatter(X_test, y_test, color="black", label='Data points')
plt.plot(X_test, y_pred, color="blue", linewidth=2, label=r'Linear Regression')
plt.plot(X_test, line_y_ransac, color="red", linewidth=2, label=r'RANSAC Regressor')


plt.legend(loc='best')
plt.xlabel('AveRooms')
plt.ylabel('MedHouseVal')
plt.ylim((0,5))
plt.xlim((0,10))
plt.title("Testing data")
plt.show()


## Polynomial Regression


Polynomial Regression assumes that the function to be fitted is a polynomial function.

This example demonstrates the problems of underfitting and overfitting and how we can use linear regression with polynomial features to approximate nonlinear functions. The plot shows the function that we want to approximate, which is a part of the cosine function. In addition, the samples from the real function and the approximations of different models are displayed. The models have polynomial features of different degrees. We can see that a linear function (polynomial with degree 1) is not sufficient to fit the training samples. This is called **underfitting**, i.e., a model is too simplistic to capture the
underlying patterns in the data. A polynomial of degree 4 approximates the true function almost perfectly. However, for higher degrees the model will **overfit** the training data, i.e., a model performs
exceptionally well on the training data but fails to generalize to
new, unseen data. We evaluate quantitatively **overfitting** / **underfitting** by using cross-validation. We calculate the mean squared error (MSE) on the validation set, the higher, the less likely the model generalizes correctly from the training data.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.model_selection import cross_val_score

In [ ]:
def true_fun(X):
    return np.cos(1.5 * np.pi * X)

np.random.seed(0)

n_samples = 30
degrees = [1, 4, 15]

X = np.sort(np.random.rand(n_samples))
y = true_fun(X) + np.random.randn(n_samples) * 0.1

plt.figure(figsize=(14, 5))
for i in range(len(degrees)):
    ax = plt.subplot(1, len(degrees), i + 1)
    plt.setp(ax, xticks=(), yticks=())

    polynomial_features = PolynomialFeatures(degree=degrees[i],
                                             include_bias=False)
    linear_regression = linear_model.LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    pipeline.fit(X[:, np.newaxis], y)

    # Evaluate the models using crossvalidation
    scores = cross_val_score(pipeline, X[:, np.newaxis], y,
                             scoring="neg_mean_squared_error", cv=10)

    X_test = np.linspace(0, 1, 100)
    plt.plot(X_test, pipeline.predict(X_test[:, np.newaxis]), label="Model")
    plt.plot(X_test, true_fun(X_test), label="True function")
    plt.scatter(X, y, edgecolor='b', s=20, label="Samples")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim((0, 1))
    plt.ylim((-2, 2))
    plt.legend(loc="best")
    plt.title("Degree {}\nMSE = {:.2e}(+/- {:.2e})".format(
        degrees[i], -scores.mean(), scores.std()))

# plt.savefig('./figures/polynomial_regresion.png')
plt.show()

## Polynomial Ridge Regression

Ridge regression is a regularization technique that can be utilized by maintaining a loss function of the following form.

$Loss = MSE + \lambda ||w||^2_2$

This leads to a parameter estimator equations,

$\hat{\beta}_{ridge} = (X^T X + \lambda I_p)^{-1}X^T y$

It can be understood from the example below that ridge regression controls the overfitting problem, as the degree 15 polynomial fits the ground truth reasonably well.

In [ ]:
def true_fun(X):
    return np.cos(1.5 * np.pi * X)

np.random.seed(0)

n_samples = 30
degrees = [1, 4, 15]
alpha = 0.0001 # lambda in the slides

X = np.sort(np.random.rand(n_samples))
y = true_fun(X) + np.random.randn(n_samples) * 0.1

plt.figure(figsize=(14, 5))
for i in range(len(degrees)):
    ax = plt.subplot(1, len(degrees), i + 1)
    plt.setp(ax, xticks=(), yticks=())

    polynomial_features = PolynomialFeatures(degree=degrees[i],
                                             include_bias=False)
    ridge = linear_model.Ridge(alpha=alpha, fit_intercept=True)
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("ridge_regresssion", ridge)])
    pipeline.fit(X[:, np.newaxis], y)

    # Evaluate the models using crossvalidation
    scores = cross_val_score(pipeline, X[:, np.newaxis], y,
                             scoring="neg_mean_squared_error", cv=10)

    X_test = np.linspace(0, 1, 100)
    plt.plot(X_test, pipeline.predict(X_test[:, np.newaxis]), label="Model")
    plt.plot(X_test, true_fun(X_test), label="True function")
    plt.scatter(X, y, edgecolor='b', s=20, label="Samples")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim((0, 1))
    plt.ylim((-2, 2))
    plt.legend(loc="best")
    plt.title("Degree {}\nMSE = {:.2e}(+/- {:.2e})".format(
        degrees[i], -scores.mean(), scores.std()))

# plt.savefig('./figures/polynomial_ridge_regresion.png')
plt.show()

## Ridge Coefficients as a Function of the Regularization

This example also shows the usefulness of applying Ridge regression to highly ill-conditioned matrices. For such matrices, **a slight change in the target variable can cause huge variances in the calculated weights**. In such cases, it is useful to set a certain regularization (alpha) to reduce this variation (noise).

When alpha is very large, the regularization effect dominates the squared loss function and the coefficients tend to zero. At the end of the path, as alpha tends toward zero and the solution tends towards the ordinary least squares, coefficients exhibit big oscillations. In practise it is necessary to tune alpha in such a way that a balance is maintained between both.

In [ ]:
# Author: Fabian Pedregosa -- <fabian.pedregosa@inria.fr>
# License: BSD 3 clause


# X is the 10x10 Hilbert matrix
X = 1. / (np.arange(1, 11) + np.arange(0, 10)[:, np.newaxis])
y = np.ones(10)

# #############################################################################
# Compute paths

n_alphas = 200
alphas = np.logspace(-10, -2, n_alphas)

coefs = []
for a in alphas:
    ridge = linear_model.Ridge(alpha=a, fit_intercept=False)
    ridge.fit(X, y)
    coefs.append(ridge.coef_)

# #############################################################################
# Display results

ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.xlabel('$\lambda$')
plt.ylabel('weights')
plt.title('Ridge coefficients as a function of the regularization')
plt.axis('tight')
# plt.savefig('./figures/redge_coeff_regularization.png')
plt.show()

## LASSO Regression

Lasso regression is a regularization technique that can be utilized by maintaining a loss function of the following form.

$Loss = MSE + \lambda ||w||_1$

Compare this with the ridge regression equation.

 Lasso regression is generally used in cases where it is preferable to have a sparse representation of the model. This is illustrated in the example below, where using Lasso regression, many coefficients are forced to be zero
 .

In [ ]:
from sklearn.linear_model import  Lasso
from sklearn.datasets import make_regression

# Generate synthetic data with informative and irrelevant features
X, y, true_coefficients = make_regression(
    n_samples=100, n_features=50, n_informative=10, noise=20.5, coef=True, random_state=42
)

# Create a Lasso regression model
lasso_model = Lasso(alpha=3)  # Alpha is the regularization strength

# Fit the Lasso regression model to the data
lasso_model.fit(X, y)

# Create a Linear Regression model
simple_linear_model = linear_model.LinearRegression()

# Fit the Linear Regression model to the data
simple_linear_model.fit(X, y)

# Plot the true coefficients and the estimated coefficients for Lasso and Linear Regression
plt.figure(figsize=(10, 6))

plt.plot(true_coefficients, label='True Coefficients', marker='o')
plt.plot(lasso_model.coef_, label='Lasso Regression', marker='x')
plt.plot(simple_linear_model.coef_, label='Linear Regression', marker='s')

plt.xlabel('Coefficient Index')
plt.ylabel('Coefficient Value')
plt.title('True vs. Estimated Coefficients')
plt.legend()
plt.grid()
plt.show()



print("Number of non-zero coefficients (Linear Regression):", np.sum(simple_linear_model.coef_ != 0))
print("Number of non-zero coefficients (Lasso Regression):", np.sum(lasso_model.coef_ != 0))

In [ ]:
plt.figure(figsize=(12, 6))

# Plot the true coefficients as stem plot
plt.stem(range(len(true_coefficients)), true_coefficients, markerfmt='go', linefmt='g', basefmt=' ')

# Plot the Lasso coefficients as stem plot with a small offset
plt.stem(np.arange(len(true_coefficients)) + 0.3, lasso_model.coef_, markerfmt='rx', linefmt='r', basefmt=' ')

# Plot the Linear Regression coefficients as stem plot with a larger offset
plt.stem(np.arange(len(true_coefficients)) + 0.5, simple_linear_model.coef_, markerfmt='bs', linefmt='b', basefmt=' ')

plt.xlabel('Coefficient Index')
plt.ylabel('Coefficient Value')
plt.title('True vs. Estimated Coefficients')
plt.legend(['True Coefficients', 'Lasso Regression', 'Linear Regression'])
plt.grid()
plt.show()

## Multi Variable Linear regression

Review the code given below on multivariable linear regression

In [ ]:
#load the dataset
from sklearn.datasets import fetch_california_housing

# Load the diabetes dataset
data_X, data_y = fetch_california_housing(return_X_y=True)

#remember index starts from 0
# Split the data into training/testing sets
X_train, X_test, y_train, y_test = model_selection.train_test_split(data_X, data_y, test_size=0.2, random_state=0)

X_train, X_val, y_train, y_val  = model_selection.train_test_split(X_train, y_train, test_size=0.25, random_state=0)

print("Total dataset elements",data_X.shape)
print("Train dataset elements",X_train.shape)
print("Validatiom dataset elements",X_val.shape)
print("Test dataset elements",X_test.shape)

Using sklearn to fit the parameters and find the regression coefficients.

In [ ]:
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The intercept
print("Coefficients: \n", regr.intercept_)

y_pred = regr.predict(X_test)



# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

#see the impact of each variable

regr_coef=regr.coef_/np.max(regr.coef_)
print("Coefficients: \n", regr_coef)
#Here, value=1 is the most relevant feature.
california_housing = fetch_california_housing(as_frame=True)
california_housing.data.head()

Calculating p values

In [ ]:
import pandas as pd
import statsmodels.api as sm

# Assuming you have already fitted the linear regression model (regr.fit(X_train, y_train))

# Add a constant column to the feature matrix (required by statsmodels)
X_train_with_constant = sm.add_constant(X_train)

# Fit the OLS (Ordinary Least Squares) model
ols_model = sm.OLS(y_train, X_train_with_constant).fit()

# Get summary statistics of the model
summary = ols_model.summary()

# Extract p-values from the summary for all features
p_values = summary.tables[1].data[1:]

# Create a DataFrame to associate p-values with feature names
p_values_df = pd.DataFrame(p_values, columns=['Feature', 'Coefficient', 'Standard Error', 't-value', 'P-Value', 'Lower CI', 'Upper CI'])
p_values_df['P-Value'] = p_values_df['P-Value'].astype(float)


print(summary)
print(p_values_df)


Calculating Standard errors and t-values

In [ ]:
from scipy import stats
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)
params = np.append(lm.intercept_,lm.coef_)
predictions = lm.predict(X_train)

newX = pd.DataFrame({"Constant":np.ones(len(X_train))}).join(pd.DataFrame(X_train))
MSE = (sum((y_train-predictions)**2))/(len(newX)-len(newX.columns))

# Note if you don't want to use a DataFrame replace the two lines above with
# newX = np.append(np.ones((len(X),1)), X, axis=1)
# MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
sd_b = np.sqrt(var_b)
ts_b = params/ sd_b



sd_b = np.round(sd_b,3)
ts_b = np.round(ts_b,3)

params = np.round(params,4)

myDF3 = pd.DataFrame()
myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"] = [params,sd_b,ts_b]
print(myDF3)